Imports

In [59]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from ratelimit import limits, sleep_and_retry
from pathlib import Path
import os.path

In [60]:
grades_exist = os.path.exists(os.path.join(Path().resolve(), "grades.csv"))
reviews_exist = os.path.exists(os.path.join(Path().resolve(), "reviews.csv"))

In [61]:
if reviews_exist == False:
        reviews = []
        done = False
        offset = 0

        while done == False:
                r = requests.get("https://planetterp.com/api/v1/professors", params = {"offset":offset, "reviews": "true", "limit":100},)
                if r.json() == []:
                        done = True
                else:
                        reviews.append(r.json())
                        offset = offset+100
                        #print(offset)
    
#print(len(reviews))
# expect 12200

In [62]:
if reviews_exist == False:

    df = pd.DataFrame()

    count = 0

    for i in reviews:
        for j in i:
            if j.get("reviews") != []:
                for k in j.get("reviews"):
                    if k.get("course") != None and j.get("type") == "professor":
                        df.at[count, "name"] = j.get("name")
                        df.at[count, "slug"] = j.get("slug")
                        df.at[count, "type"] = j.get("type")
                        df.at[count, "course"] = k.get("course")
                        df.at[count, "rating"] = k.get("rating")
                        df.at[count, "review"] = k.get("review")
                        df.at[count, "date"] = k.get("created")[:10]
                        count = count + 1

    df

In [63]:
if grades_exist == False:

    grades = []

    professors = df["name"].drop_duplicates()

    for prof in professors:
        r = requests.get("https://planetterp.com/api/v1/grades", params = {"offset":offset, "reviews": "true", "limit":100, "professor": prof})
        grades.append(r.json())
        #print(r.json())

In [64]:
if grades_exist == False:

    grade_df = pd.DataFrame()

    count = 0

    for i in grades:
        if i != []:
            for j in i:
                grade_df.at[count, "professor"] = j.get("professor")
                grade_df.at[count, "course"] = j.get("course")
                grade_df.at[count, "semester"] = j.get("semester")
                grade_df.at[count, "section"] = j.get("section")
                grade_df.at[count, "A+"] = j.get("A+")
                grade_df.at[count, "A"] = j.get("A")
                grade_df.at[count, "A-"] = j.get("A-")
                grade_df.at[count, "B+"] = j.get("B+")
                grade_df.at[count, "B"] = j.get("B")
                grade_df.at[count, "B-"] = j.get("B-")
                grade_df.at[count, "C+"] = j.get("C+")
                grade_df.at[count, "C"] = j.get("C")
                grade_df.at[count, "C-"] = j.get("C-")
                grade_df.at[count, "D+"] = j.get("D+")
                grade_df.at[count, "D"] = j.get("D")
                grade_df.at[count, "D-"] = j.get("D-")
                grade_df.at[count, "F"] = j.get("F")
                grade_df.at[count, "W"] = j.get("W")
                grade_df.at[count, "Other"] = j.get("Other")
                count = count + 1
                print(j.get("professor"))

    grade_df

In [65]:
if reviews_exist == False:
    df.to_csv("reviews.csv", encoding='utf-8', index=False)
if grades_exist == False:
    grade_df.to_csv("grades.csv", encoding = "utf-8", index = False)

In [66]:
ratings_df = pd.read_csv("reviews.csv")
ratings_df

,name,slug,type,course,rating,review,date
0,Abhijit Dasgupta,dasgupta_abhijit,professor,ENAE202,5.0,I took the ENAE version of this class but it i...,2019-12-16
1,Abhijit Dasgupta,dasgupta_abhijit,professor,ENME473,5.0,"The man. Really excellent lecturer, he puts in...",2022-01-21
2,Abhijit Dasgupta,dasgupta_abhijit,professor,ENME473,3.0,Dasgupta is very passionate about this subject...,2022-12-12
3,Abigail McEwen,mcewen,professor,ARTH250,2.0,"Prof. McEwan seems fair and nice enough, but I...",2010-11-18
4,Abigail McEwen,mcewen,professor,ARTH260,1.0,I was excited to take this class because of th...,2020-11-18
...,...,...,...,...,...,...,...
19180,Toby Lin,lin,professor,HACS101,2.0,Super unclear with just about everything. His ...,2023-03-27
19181,Toby Lin,lin,professor,HACS101,1.0,Absolutely useless teacher. Acts stuck up and ...,2023-04-24
19182,Toby Lin,lin,professor,HACS101,1.0,Manages to give a 4-credit class worth of work...,2023-04-26
19183,Toby Lin,lin,professor,HACS101,1.0,Rude and dismissive to students. Full of himse...,2023-04-27


In [67]:
grades_df = pd.read_csv("grades.csv")
grades_df

,professor,course,semester,section,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W,Other
0,Abhijit Dasgupta,ENME674,201201,0101,4.0,5.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,Abhijit Dasgupta,ENMA300,201208,0301,0.0,0.0,4.0,3.0,13.0,7.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0
2,Abhijit Dasgupta,ENMA300,201208,0302,1.0,4.0,9.0,4.0,10.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Abhijit Dasgupta,ENMA300,201208,0303,1.0,1.0,5.0,2.0,12.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,Abhijit Dasgupta,ENME684,201301,0101,2.0,6.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87241,Emma Newton,COMM107,202208,4001,9.0,2.0,6.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87242,Xingman Wu,COMM107,202208,9901,3.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87243,Xingman Wu,COMM107,202208,9909,1.0,2.0,14.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
87244,Joshua Poston,UNIV100,202208,FC60,3.0,0.0,11.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
from pathlib import Path
import os.path

temp = os.path.exists(os.path.join(Path().resolve(), "grades.csv"))

if temp:
    print("e")

e


In [69]:
r = requests.get("https://planetterp.com/api/v1/grades", params = {"offset":0, "reviews": "true", "limit":100, "course":"MATH140", "professor":"Craig Schlenoff"},)
print(r.json())

[{'course': 'MATH140', 'professor': 'Craig Schlenoff', 'semester': '201708', 'section': 'FC06', 'A+': 0, 'A': 2, 'A-': 3, 'B+': 2, 'B': 7, 'B-': 5, 'C+': 3, 'C': 3, 'C-': 5, 'D+': 0, 'D': 2, 'D-': 0, 'F': 0, 'W': 1, 'Other': 0}, {'course': 'MATH140', 'professor': 'Craig Schlenoff', 'semester': '201808', 'section': 'FC06', 'A+': 2, 'A': 1, 'A-': 5, 'B+': 2, 'B': 7, 'B-': 2, 'C+': 0, 'C': 5, 'C-': 2, 'D+': 0, 'D': 2, 'D-': 0, 'F': 4, 'W': 2, 'Other': 0}, {'course': 'MATH140', 'professor': 'Craig Schlenoff', 'semester': '201908', 'section': 'FC07', 'A+': 3, 'A': 2, 'A-': 5, 'B+': 2, 'B': 2, 'B-': 1, 'C+': 1, 'C': 3, 'C-': 0, 'D+': 0, 'D': 0, 'D-': 1, 'F': 4, 'W': 4, 'Other': 0}, {'course': 'MATH140', 'professor': 'Craig Schlenoff', 'semester': '202008', 'section': 'FC07', 'A+': 1, 'A': 1, 'A-': 2, 'B+': 2, 'B': 4, 'B-': 6, 'C+': 4, 'C': 3, 'C-': 1, 'D+': 0, 'D': 2, 'D-': 1, 'F': 2, 'W': 4, 'Other': 0}, {'course': 'MATH140', 'professor': 'Craig Schlenoff', 'semester': '202008', 'section': 

In [70]:
r = requests.get("https://planetterp.com/api/v1/professors", params = {"offset":11, "reviews": "true", "limit":1},)
print(r.json())

[{'courses': ['PHYS405', 'PHYS275', 'PHYS758E', 'PHYS273', 'PHYS273H', 'PHYS273', 'PHYS273H'], 'average_rating': 4.25, 'type': 'professor', 'reviews': [{'professor': 'Abolhassan Jawahery', 'course': 'PHYS405', 'review': "Phys 405 is difficult. It just is. Now that we've gotten that out of the way...Prof. Jawahery is great in that he explains certain concepts very well. Sometimes he might nitpick (instance in a lab report or presentation), but usually it is not to be critical in the negative sense but helpful and INSTRUCTIVE.", 'rating': 4, 'expected_grade': '', 'created': '2011-12-12T20:31:00Z'}, {'professor': 'Abolhassan Jawahery', 'course': 'PHYS273', 'review': "PHYS 273 is a challenging course. It's a lot of content in a short amount of time. That being said, Prof. Jawahery tries to make the most of it and present it to his students in a comprehensible way. His content builds on itself, so if you're paying attention along the way, you'll find content later in the course quite easy t